# Figure DGMV

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [ ]:
# Directories
dir05x = '../paper_deficit/output/05_prep_other/fig_dgvm/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# Colors
from palettable.cartocolors.qualitative import Safe_10
cpal = Safe_10
colors = [cpal.mpl_colors[i] for i in np.arange(0, len(cpal.colors))]

c_pot = colors[1]
c_mo = colors[4] 
c_walker = colors[2]
c_erb = colors[3]
c_sand = colors[0]

c_dgvm_m = '#000000'
c_dgvm = '#bdbdbd'

c_edge = '#000000'

In [ ]:
# List of DGVMs
list_dgvm = ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
             'jules', 'lpjguess', 'lpjwsl', 'orchidee']

In [ ]:
# Get data
df_g = pd.read_csv(os.path.join(dir05x, 'data_dgvm_global.csv'))
df_e = pd.read_csv(os.path.join(dir05x, 'data_dgvm_ecozone.csv'))

# Create individual dataframes for each data source
df_pot = df_g[df_g.model == 'ganzenmueller'].drop('model', axis=1)
df_erb = df_g[df_g.model == 'erb'].drop('model', axis=1)
df_mo = df_g[df_g.model == 'mo'].drop('model', axis=1)
df_walker = df_g[df_g.model == 'walker'].drop('model', axis=1)
df_sand030 = df_g[df_g.model == 'sanderman030'].drop('model', axis=1)
df_sand100 = df_g[df_g.model == 'sanderman100'].drop('model', axis=1)
df_sand200 = df_g[df_g.model == 'sanderman200'].drop('model', axis=1)
df_dgvm_m = df_g[df_g.model == 'dgvm_mean'].drop('model', axis=1)
df_dgvm = df_g[df_g.model.isin(list_dgvm)]

# Merge df_g and df_e
df_all = pd.concat([df_e, df_g.assign(cat='global')])

In [ ]:
# Prepare upper part of plot
# Lists
list_cat = ['global', 'trop1', 'trop2', 'subt', 'temp', 'bore', 'pola']
list_dgvm = ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach',
             'jules', 'lpjguess', 'lpjwsl', 'orchidee']
list_xaxis = ['ganzenmueller', 'mo', 'walker', 'erb', 'sanderman200', 'dgvm_mean']
list_xlabels = ['This study', 'Mo', 'Walker', 'Erb', 'Sanderman', 'DGVM mean']

# Helper functions
def make_df_label():

    """Make dataframe with label information"""
    
    # Define a dictionary mapping models to their labels
    labels = {
        'ganzenmueller': ('This study', 'This study (0-30 cm)', c_pot),
        'erb': ('Erb', 'NA', c_erb),
        'walker': ('Walker', 'Walker (0-200cm)', c_walker),
        'mo': ('Mo (forest biomass)', 'NA', c_mo),
        'sanderman030': ('NA', 'Sanderman (0-30 cm)', c_sand),
        'sanderman100': ('NA', 'Sanderman (0-100 cm)', c_sand),
        'sanderman200': ('NA', 'Sanderman (0-200 cm)', c_sand),
        'dgvm_mean': ('DGVM mean', 'DGVM mean', c_dgvm_m),
        'cablepop': ('CABLE-POP', 'CABLE-POP', c_dgvm),
        'classic': ('CLASSIC', 'CLASSIC', c_dgvm),
        'clm': ('CLM', 'CLM', c_dgvm),
        'dlem': ('DLEM', 'DLEM', c_dgvm),
        'ibis': ('IBIS', 'IBIS', c_dgvm),
        'isam': ('ISAM', 'ISAM', c_dgvm),
        'jsbach': ('JSBACH', 'JSBACH', c_dgvm),
        'jules': ('JULES', 'JULES', c_dgvm),
        'lpjguess': ('LPJ-GUESS', 'LPJ-GUESS', c_dgvm),
        'lpjwsl': ('LPJ-WSL', 'LPJ-WSL', c_dgvm),
        'orchidee': ('ORCHIDEE', 'ORCHIDEE', c_dgvm)
    }
    
    # Convert the labels dictionary into a DataFrame
    labels_df = pd.DataFrame.from_dict(
        labels, orient='index', 
        columns=['label_veg', 'label_soil', 'color']) \
        .reset_index()
    labels_df.rename(columns={'index': 'model'}, inplace=True)
    
    # Merge the original DataFrame with the labels DataFrame
    df_label = df_g[['model', 's3_cveg', 's3_csoil']] \
        .merge(labels_df, on='model', how='left')

    return df_label


def add_veg_label(m, ycor, xcor):

    """ Add vegetation labels"""
    
    df_label = make_df_label()
    df_sel = df_label[df_label.model == m]
    
    color = '#000000' if m in list_dgvm else df_sel.color.item()
    weight = 'normal' if m in list_dgvm else 'bold'
    
    ax01.annotate(df_sel.label_veg.item(), 
                 xy=(1 + ycor, df_sel.s3_cveg.item()+xcor), 
                 va='center', ha='left',size=8, color=color, weight=weight);


def add_soil_label(m, ycor, xcor):

    """ Add soil labels"""
    
    df_label = make_df_label()
    df_sel = df_label[df_label.model == m]

    color = '#000000' if m in list_dgvm else df_sel.color.item()
    weight = 'normal' if m in list_dgvm else 'bold'

    for ax in [ax02, ax03]:
        ax.annotate(df_sel.label_soil.item(), 
                    xy=(1 + ycor, df_sel.s3_csoil.item()+xcor), 
                    va='center', ha='left', size=8, color=color, weight=weight);

# vegetation lines
def plot_veg_lines():

    """ Plot vegetation lines"""
    
    lw=3
    # vegetation data
    ax01.plot((0, 1), (df_dgvm.s2_cveg, df_dgvm.s3_cveg),'-', color=c_dgvm, lw=lw)
    ax01.plot((0, 1), (df_erb.s2_cveg, df_erb.s3_cveg),'-', color=c_erb, lw=lw)
    ax01.plot((0, 1), (df_walker.s2_cveg, df_walker.s3_cveg),'-', color=c_walker, lw=lw)
    ax01.plot((0, 1), (df_dgvm_m.s2_cveg, df_dgvm_m.s3_cveg),'-', color=c_dgvm_m, lw=lw)
    ax01.plot((0, 1), (df_mo.s2_cveg, df_mo.s3_cveg),'-', color=c_mo, lw=lw)
    ax01.plot((0, 1), (df_pot.s2_cveg, df_pot.s3_cveg),'-', color=c_pot, lw=lw)

    # vegetation labels  
    add_veg_label('ganzenmueller', 0.05, +8)
    add_veg_label('erb', 0.05, +6)
    add_veg_label('mo', 0.05, -8)
    add_veg_label('walker', 0.05, +12)
    add_veg_label('dgvm_mean', 0.05, +6)
    add_veg_label('cablepop', 0.05, +2)
    add_veg_label('classic', 0.05, -6)
    add_veg_label('clm', 0.05, +5)
    add_veg_label('dlem', 0.05, +7)
    add_veg_label('ibis', 0.05, 0)
    add_veg_label('isam', 0.05, 0)
    add_veg_label('jsbach', 0.05, -7)
    add_veg_label('jules', 0.05, +19)
    add_veg_label('lpjguess', 0.05, -16)
    add_veg_label('lpjwsl', 0.05, -8)
    add_veg_label('orchidee', 0.05, -10)

    
def plot_soil_lines():

    """Plot soil lines"""
    
    lw=3
    for i in [ax02, ax03]:
        # soil
        i.plot((0, 1), (df_dgvm.s2_csoil, df_dgvm.s3_csoil),'-', color=c_dgvm, lw=lw)
        i.plot((0, 1), (df_sand030.s2_csoil, df_sand030.s3_csoil),'-', color=c_sand, lw=lw)
        i.plot((0, 1), (df_sand100.s2_csoil, df_sand100.s3_csoil),'-', color=c_sand, lw=lw)
        i.plot((0, 1), (df_sand200.s2_csoil, df_sand200.s3_csoil),'-', color=c_sand, lw=lw)
        i.plot((0, 1), (df_walker.s2_csoil, df_walker.s3_csoil),'-', color=c_walker, lw=lw)
        i.plot((0, 1), (df_dgvm_m.s2_csoil, df_dgvm_m.s3_csoil),'-', color=c_dgvm_m, lw=lw)
        i.plot((0, 1), (df_pot.s2_csoil, df_pot.s3_csoil),'-', color=c_pot, lw=lw)
    
    # soil labels  
    add_soil_label('ganzenmueller', 0.05, 0)
    add_soil_label('sanderman030', 0.05, +30)
    add_soil_label('sanderman100', 0.05, +60)
    add_soil_label('sanderman200', 0.05, +60)
    add_soil_label('walker', 0.05, -70)
    add_soil_label('dgvm_mean', 0.05, 4)
    add_soil_label('cablepop', 0.05, 0)
    add_soil_label('classic', 0.05, 0)
    add_soil_label('clm', 0.05, +70)
    add_soil_label('dlem', 0.05, 0)
    add_soil_label('ibis', 0.05, -10)
    add_soil_label('isam', 0.05, -70)
    add_soil_label('jsbach', 0.05, 0)
    add_soil_label('jules', 0.05, -70)
    add_soil_label('lpjguess', 0.05, 0)
    add_soil_label('lpjwsl', 0.05, +10)
    add_soil_label('orchidee', 0.05, -5)

    
def plot_layout(ax):

    """ Define layout settings"""
    
    linewidth = 2
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['bottom'].set_visible(False)
    ax.xaxis.set_tick_params(bottom=False)
    ax.yaxis.set_tick_params(width=linewidth)
    ax.set_xlim(-0.25, 1.25)
    ax.set_xticks([0,1])

In [ ]:
# Plot
fig = plt.figure(figsize=(9, 7), dpi=600)
fig.subplots_adjust(left=0.08, bottom=0.03, right=0.97, top=0.97)  # adjust space between axes

gs = fig.add_gridspec(3, 3, 
                      hspace=0.25, 
                      wspace=0.25,
                      height_ratios=(0.6, 0.2, 0.2),
                      width_ratios=(0.33, 0.33, 0.33))
# Axis upper plot
ax0 = fig.add_subplot(gs[0,:], zorder=3)
ax01 =fig.add_axes([0.08,.55,.3,.42], zorder=2)

ax02 =fig.add_axes([0.55,.85,.3,.12], zorder=2)
ax03 =fig.add_axes([0.55,.55,.3,.29], zorder=2)

# Axis lower plot
ax1 = fig.add_subplot(gs[1,0])
ax2 = fig.add_subplot(gs[1,1])
ax3 = fig.add_subplot(gs[1,2])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])
ax6 = fig.add_subplot(gs[2,2])


# Plot a
plot_veg_lines()
ax01.set_ylabel('AGBC + BGBC (PgC)')
ax01.set_xticks([0, 1])
ax01.set_xticklabels(['Potential\n(S2)', 'Actual\n(S3)'])
plot_layout(ax01)


# Plot b
plot_soil_lines()
ax02.set_ylim(1450, 3500)
ax02.set_xticks([0, 1])
ax02.set_xticklabels(['', ''])
ax03.set_ylim(400, 1350)
ax03.set_ylabel('                     SOC (PgC)')
ax03.set_xticks([0, 1])
ax03.set_xticklabels(['Potential\n(S2)', 'Actual\n(S3)'])
plot_layout(ax02)
plot_layout(ax03)


# Plot bar
for ax in [ax4, ax5, ax6]:
    ax.set_ylim(-5, 65)
    ax.axhline(0, color='#000000', linewidth = 2)   
for ax in [ax2, ax3]:
    ax.set_ylim(-9, 120)
    ax.axhline(0, color='#000000', linewidth = 2)
    
for i in range(0, 6):
    
    ax = [ax1, ax2, ax3, ax4, ax5, ax6][i]
    
    cat = ['global', 'trop1', 'trop2', 'subt', 'temp', 'bore'][i] # polar not shown
    subtitle = ['Global', 'Tropical Rainforest', 'Other Tropical', 
                'Subtropical', 'Temperate', 'Boreal'][i]
    df_sel = df_all[(df_all.cat == cat)]
 
    ax.bar(0, (df_sel[df_sel.model == 'dgvm_mean'].cveg_d * -1), width=0.2, 
           color=c_dgvm_m, edgecolor=c_edge)
    ax.bar(0+0.2, (df_sel[df_sel.model == 'ganzenmueller'].cveg_d * -1), width=0.2, 
           color=c_pot, edgecolor=c_edge)
    ax.bar(0+0.4, (df_sel[df_sel.model == 'mo'].cveg_d * -1), width=0.2, 
           color=c_mo, edgecolor=c_edge)
    ax.bar(0+0.6, (df_sel[df_sel.model == 'walker'].cveg_d * -1), width=0.2, 
           color=c_walker, edgecolor=c_edge)
    ax.bar(0+0.8, (df_sel[df_sel.model == 'erb'].cveg_d * -1), width=0.2, 
           color=c_erb, edgecolor=c_edge)

    ax.bar(1.2, (df_sel[df_sel.model == 'dgvm_mean'].csoil_d * -1), width=0.2, 
           color=c_dgvm_m, edgecolor=c_edge)
    ax.bar(1.2+0.2, (df_sel[df_sel.model == 'ganzenmueller'].csoil_d * -1), width=0.2, 
           color=c_pot, edgecolor=c_edge)
    ax.bar(1.2+0.4, (df_sel[df_sel.model == 'walker'].csoil_d * -1), width=0.2, 
           color=c_walker, edgecolor=c_edge)
    ax.bar(1.2+0.6, (df_sel[df_sel.model == 'sanderman200'].csoil_d * -1), width=0.2, 
           color=c_sand, edgecolor=c_edge)
    ax.bar(1.2+0.6, (df_sel[df_sel.model == 'sanderman100'].csoil_d * -1), width=0.2, 
           color=c_sand, edgecolor=c_edge)
    ax.bar(1.2+0.6, (df_sel[df_sel.model == 'sanderman030'].csoil_d * -1), width=0.2, 
           color=c_sand, edgecolor=c_edge)

    df_dgvm_min = df_all[(df_all.cat == cat) & (df_all.model.isin(list_dgvm))].min()
    df_dgvm_max = df_all[(df_all.cat == cat) & (df_all.model.isin(list_dgvm))].max()
    ax.vlines(0, df_dgvm_min.cveg_d * -1, df_dgvm_max.cveg_d * -1, color=c_dgvm, lw=2)
    ax.vlines(1.2, df_dgvm_min.csoil_d * -1, df_dgvm_max.csoil_d * -1, color=c_dgvm, lw=2)    
    
    df_dgvm_x = df_all[(df_all.cat == cat) & (df_all.model.isin(list_dgvm))]

    for ii in range(0, len(df_dgvm_x)):
        ax.hlines((df_dgvm_x.iloc[ii].cveg_d * -1), -0.05, 0.05, color=c_dgvm, lw=1)
        ax.hlines((df_dgvm_x.iloc[ii].csoil_d * -1), 1.15, 1.25, color=c_dgvm, lw=1)
    
    ax.set_title(subtitle)
    ax.axvline(1.0, color='#000000', ls=':', linewidth = 2)    
    
    ax.set_xticks([0.3, 1.3])
    ax.set_xticklabels('', ha='center')
    ax.tick_params(left = True, bottom = False)

    linewidth = 2
    ax.spines['right'].set_visible(False) # remove right plot boundary
    ax.spines['top'].set_visible(False) # remove top plot boundary
    ax.spines['left'].set_linewidth(linewidth) # make left axis thicker
    ax.spines['bottom'].set_linewidth(linewidth) # make bottom axis thicker
    ax.xaxis.set_tick_params(width=linewidth) # make x-axis tick marks thicker
    ax.yaxis.set_tick_params(width=linewidth) # make y-axis tick marks thicker

    
ax1.set_ylim(None, 370)
ax1.text(0.31, 350, 'AGBC + BGBC', ha='center', va='center')
ax1.text(1.175, 350, 'SOC', ha='center', va='center')

#ax1.text(0+0.36, 10, 'forests only', rotation=90, ha='center', va='bottom', size='small', c='white')
#ax1.text(0+0.38, 200, '*', rotation=90, ha='center', va='bottom', size='large', c=c_mo)
ax1.text(1.2, 100, 'various', rotation=90, ha='center', va='bottom', size='small', c=c_dgvm)
ax1.text(1.2+0.205, 80, '0-30 cm', rotation=90, ha='center', va='bottom', size='small', c=c_pot)
ax1.text(1.2+0.410, 120, '0-200 cm', rotation=90, ha='center', va='bottom', size='small', c=c_walker)
ax1.text(1.2+0.615, 100, '0-30|-100|-200 cm', rotation=90, ha='center', va='bottom', size='small', c=c_sand)

ax1.set_ylabel('Carbon deficit (PgC)                                   ')

ax0.axis('off')
ax0.set_xlim(0, 1)
ax0.set_ylim(0, 1)
#ax0.set_ylabel('Test')
ax0.patch.set_alpha(0)
ax0.plot((0.521, 0.535), (0.740, 0.760), lw=2, c='#000000')
ax0.plot((0.521, 0.535), (0.727, 0.747), lw=2, c='#000000')

fig.text(0.02, 0.97, 'A', fontsize = 'xx-large', ha='center', va='center', fontweight ="bold")
fig.text(0.48, 0.97, 'B', fontsize = 'xx-large', ha='center', va='center', fontweight ="bold")
fig.text(0.02, 0.45, 'C', fontsize = 'xx-large', ha='center', va='center', fontweight ="bold");

# Export
plt.savefig(os.path.join(dir06, 'pdf/fig05_dgvm.pdf'), bbox_inches='tight', dpi=600)
plt.savefig(os.path.join(dir06, 'png/fig05_dgvm.png'), bbox_inches='tight', dpi=600);